In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/_z/99vh6s_13zlf23p512dzc53r0000gn/T/ipykernel_19239/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [15]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/finko/Desktop/Ironhack/lab-natural-language-processing/data/kg_train.csv", encoding='latin-1')


# Reduce the training set to speed up development. 
data = data.head(1000)

### Let's divide the training and test set into two partitions

In [18]:
from sklearn.model_selection import train_test_split

X = data['text']
y = data['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Data Preprocessing

In [19]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [20]:
import re

def clean_html(text):
    # 1. Remove inline JavaScript/CSS
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)

    # 2. Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)

    # 3. Remove all remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [21]:
import re
import string

def full_clean_text(text):
    # Remove HTML content
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<[^>]+>', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)       # remove special characters
    text = re.sub(r'\d+', ' ', text)               # remove numbers

    # Remove single characters
    text = re.sub(r'\b[a-zA-Z]\b', '', text)       # remove single characters
    text = re.sub(r'^[a-zA-Z]\s+', '', text)       # remove single chars at start

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove leading 'b' if present (from byte conversion artifacts)
    text = re.sub(r'^b\s+', '', text)

    # Lowercase
    text = text.lower()

    return text.strip()

## Now let's work on removing stopwords
Remove the stopwords.

In [22]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

stop_words = ENGLISH_STOP_WORDS

def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return " ".join(filtered)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [27]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# One-time download (run once)
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional for better lemma coverage

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /Users/finko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/finko/nltk_data...


In [28]:
def lemmatize_text(text):
    words = text.split()
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [30]:
from collections import Counter

# Separate messages by label
ham_messages = data[data['label'] == 0]['text']
spam_messages = data[data['label'] == 1]['text']

# Simple tokenizer and cleaning
def tokenize(texts):
    all_words = []
    for text in texts:
        text = full_clean_text(text)  # reuse your cleaning function
        words = text.split()
        all_words.extend(words)
    return all_words

# Tokenize each group
ham_words = tokenize(ham_messages)
spam_words = tokenize(spam_messages)

# Count frequencies
ham_freq = Counter(ham_words).most_common(10)
spam_freq = Counter(spam_words).most_common(10)

# Display
print("Top 10 words in HAM:")
for word, freq in ham_freq:
    print(f"{word}: {freq}")

print("\nTop 10 words in SPAM:")
for word, freq in spam_freq:
    print(f"{word}: {freq}")

Top 10 words in HAM:
the: 1780
to: 1065
and: 838
of: 799
in: 616
that: 415
is: 387
for: 374
on: 328
you: 311

Top 10 words in SPAM:
the: 7048
to: 5594
of: 4982
and: 3979
in: 3291
you: 3237
this: 2674
my: 2146
your: 2071
for: 2039


## Extra features

In [32]:
# Create data_train and data_val DataFrames from X_train and X_val
data_train = pd.DataFrame({'preprocessed_text': X_train, 'label': y_train})
data_val = pd.DataFrame({'preprocessed_text': X_val, 'label': y_val})

# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,preprocessed_text,label,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,1,0,106
535,I have not been able to reach oscar this am. W...,0,1,0,101
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,1,0,141
557,I can have it announced here on Monday - can't...,0,1,0,52
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,1,1,1750


## How would work the Bag of Words with Count Vectorizer concept?

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

# Use preprocessed_text from data_train and data_val
X_train_processed = data_train['preprocessed_text']
X_val_processed = data_val['preprocessed_text']

vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_bow = vectorizer.fit_transform(X_train_processed)

# Transform the validation set (don't fit again!)
X_val_bow = vectorizer.transform(X_val_processed)

print("Shape of BoW feature matrix:", X_train_bow.shape)
print("Vocabulary sample:", list(vectorizer.vocabulary_.items())[:10])

Shape of BoW feature matrix: (800, 23374)
Vocabulary sample: [('regards', 17249), ('mr', 14191), ('nelson', 14540), ('smith', 18584), ('kindly', 12565), ('reply', 17410), ('me', 13691), ('on', 15237), ('my', 14296), ('private', 16498)]


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_processed)

# Transform the validation set
X_val_tfidf = tfidf_vectorizer.transform(X_val_processed)

# Print shape
print("TF-IDF train shape:", X_train_tfidf.shape)
print("TF-IDF val shape:", X_val_tfidf.shape)

TF-IDF train shape: (800, 23374)
TF-IDF val shape: (200, 23374)


## And the Train a Classifier?

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train the model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# Predict on validation set
y_pred = clf.predict(X_val_tfidf)

# Evaluate
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))

Accuracy: 0.95

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96       125
           1       0.97      0.89      0.93        75

    accuracy                           0.95       200
   macro avg       0.95      0.94      0.95       200
weighted avg       0.95      0.95      0.95       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# TF-IDF Vectorizer with tuned parameters
vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words='english', max_df=0.9, min_df=5)

# Vectorize
X_train_vec = vectorizer.fit_transform(X_train_processed)
X_val_vec = vectorizer.transform(X_val_processed)

# Multinomial Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
y_pred = nb.predict(X_val_vec)

# Evaluation
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nReport:\n", classification_report(y_val, y_pred))

Accuracy: 0.985

Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99       125
           1       0.97      0.99      0.98        75

    accuracy                           0.98       200
   macro avg       0.98      0.99      0.98       200
weighted avg       0.99      0.98      0.99       200

